In [4]:
import cv2
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def load_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

def detect_edges(image):
    # convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # apply gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # apply canny edge detection
    edges = cv2.Canny(blur, 50, 150)
    return edges

def filter_image(image, filter_type):
    # apply image filter
    if filter_type == 'blur':
        filtered_image = cv2.blur(image, (5, 5))
    elif filter_type == 'median':
        filtered_image = cv2.medianBlur(image, 5)
    elif filter_type == 'bilateral':
        filtered_image = cv2.bilateralFilter(image, 9, 75, 75)
    else:
        print('Invalid filter type.')
        return None

    return filtered_image

def process_images(images, filter_type):
    with ThreadPoolExecutor(max_workers=5) as executor:
        edge_futures = [executor.submit(detect_edges, image) for image in images]
        filter_futures = [executor.submit(filter_image, image, filter_type) for image in images]

        edge_images = [future.result() for future in edge_futures]
        filtered_images = [future.result() for future in filter_futures]

    return edge_images, filtered_images

def main():
    folder_path = './PNGImages'
    images = load_images(folder_path)
    filter_type = 'bilateral' # you can change the filter type

    edge_images, filtered_images = process_images(images, filter_type)

    # choose an image to display
    choice = int(input('Enter the index of the image you want to display (0-based): '))
    cv2.imshow('Original', images[choice])
    cv2.imshow('Edges', edge_images[choice])
    cv2.imshow('Filtered', filtered_images[choice])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()